***Total: 42 points***

Complete this homework by writing R code to complete the following tasks. Keep in mind:

i. Empty chunks have been included where code is required
ii. This homework requires use of data files:

  - `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg` (Problems 1, 2)
  - `GIAB_highconf_v.3.3.2.vcf.gz` (Problem 3)
  
iv. You will be graded on your code and output results. The assignment is worth 42 points total; partial credit can be awarded.

For additional resources, please refer to these links:  
Problems 1 & 2:  
  - https://www.bioconductor.org/packages/devel/bioc/vignettes/plyranges/inst/doc/an-introduction.html
  - https://bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html  
Problem 3:  
  - https://bioconductor.org/packages/release/bioc/vignettes/Rsamtools/inst/doc/Rsamtools-Overview.pdf  
Problem 4: 
  - https://bioconductor.org/packages/release/bioc/vignettes/VariantAnnotation/inst/doc/VariantAnnotation.pdf  

# Problem 1: Overlaps between genomic regions and copy number alterations. (14 points total)

### Preparation
Load copy number segment results as shown in *2.1 BED format* of *Lecture16_GenomicData.Rmd*. You will use the same file as in the lecture notes, `BRCA.genome_wide_snp_6_broad_Level_3_scna.seg`. Here is code to get you started.

In [2]:
#load packages
suppressPackageStartupMessages({
    library(tidyverse)
    library(GenomicRanges)
    library(plyranges)
    library(VariantAnnotation)
})

In [3]:
segs <- read.delim("BRCA.genome_wide_snp_6_broad_Level_3_scna.seg", as.is = TRUE)
mode(segs$Chromosome) <- "character"
segs[segs$Chromosome == 23, "Chromosome"] <- "X"
segs.gr <- as(segs, "GRanges")

### a. Find the segments in `segs.gr` that have *any* overlap with the region `chr8:128,746,347-128,755,810` (4 points)
Print out the first five unique TCGA IDs.

In [ ]:
# create GRanges object for that range
grange <- GRanges(seqnames = "8",
                    ranges = IRanges(start = 128746347, end = 128755810))

segs.overlap <- find_overlaps(segs.gr, grange)
print("First five unique TCGA IDs:")
print(segs.overlap$Sample[1:5])

[1] "First five unique TCGA IDs:"
[1] "TCGA-3C-AAAU-10A-01D-A41E-01" "TCGA-3C-AAAU-01A-11D-A41E-01"
[3] "TCGA-3C-AALI-10A-01D-A41E-01" "TCGA-3C-AALI-01A-11D-A41E-01"
[5] "TCGA-3C-AALJ-10A-01D-A41E-01"


### b. Find the mean of the `Segment_Mean` values for copy number segments that have *any* overlap with the region chr17:37,842,337-37,886,915. (4 points)

In [ ]:
# create GRanges object for that range
grange <- GRanges(seqnames = "17",
                    ranges = IRanges(start = 37842337, end = 137886915))

# find overlap
segs.overlap <- find_overlaps(segs.gr, grange)

print("Mean of Segment_Mean values:")
print(mean(segs.overlap$Segment_Mean))

[1] "Mean of Segment_Mean values:"
[1] 0.2577407


### c. Find the patient sample distribution of copy number for `PIK3CA` (hg19). (6 points)
Find the counts of samples with deletion (D; `Segment_Mean < -0.3`), neutral (N; `Segment_Mean >= -0.3 & Segment_Mean <= 0.3`), gain (G; `Segment_Mean > 0.3`) segments that have `any` overlap with `PIK3CA` gene coordinates.  


In [48]:
# the PIK3CA gene is located on hg19 chr3:178,866,145-178,957,881 (https://genome.ucsc.edu/cgi-bin/hgGene?showAllCtdRef=Y&db=hg19&hgg_gene=PIK3CA)

pik3ca_grange <- GRanges(seqnames = "3",
                    ranges = IRanges(start = 178866145, end = 178957881))

pik3ca_overlap <- find_overlaps(segs.gr, pik3ca_grange)
pik3ca_overlap %>%
    as_tibble() %>%
    mutate(mutation_type = case_when( # solution from https://stackoverflow.com/questions/36003699/how-can-i-create-a-new-column-based-on-conditional-statements-and-dplyr
        Segment_Mean < -0.3 ~ "D",
        Segment_Mean >= -0.3 & Segment_Mean <= 0.3 ~ "N",
        Segment_Mean > 0.3 ~ "G"
    )) %>%
    group_by(mutation_type) %>%
    summarize(count = n())

mutation_type,count
<chr>,<int>
D,17
G,185
N,2026


# Problem 2: Frequency of copy number alteration events within genomic regions. (12 points total) 

This problem will continue to use the copy number data stored in `segs.gr`.

### a. Create a genome-wide tile of 1Mb windows for the human genome (`hg19`). (6 points)
See *3.1 Tiling the genome* of *Lecture16_GenomicData.Rmd* for hints.


In [49]:
seqinfo <- Seqinfo(genome = "hg19")
seqinfo <- keepStandardChromosomes(seqinfo)
seqlevelsStyle(seqinfo) <- "NCBI"
slen <- seqlengths(seqinfo) # get the length of the chromosomes
tileWidth <- 1000000 # tile size of 1 Mb
tiles <- tileGenome(seqlengths = slen, tilewidth = tileWidth,
                    cut.last.tile.in.chrom = TRUE)
tiles

Warning message in (function (seqlevels, genome, new_style) :
“cannot switch some of hg19's seqlevels from UCSC to NCBI style”


GRanges object with 3114 ranges and 0 metadata columns:
         seqnames            ranges strand
            <Rle>         <IRanges>  <Rle>
     [1]        1         1-1000000      *
     [2]        1   1000001-2000000      *
     [3]        1   2000001-3000000      *
     [4]        1   3000001-4000000      *
     [5]        1   4000001-5000000      *
     ...      ...               ...    ...
  [3110]        Y 56000001-57000000      *
  [3111]        Y 57000001-58000000      *
  [3112]        Y 58000001-59000000      *
  [3113]        Y 59000001-59373566      *
  [3114]     chrM           1-16571      *
  -------
  seqinfo: 25 sequences from an unspecified genome

### b. Find the 1Mb window with the most frequent overlapping deletions. (6 points)
Find the 1Mb windows with `any` overlap with deletion copy number segments. Assume a deletion segment is defined as a segment in `segs.gr` having `Segment_Mean < -0.3`. 

Return one of the 1Mb window `Granges` entry with the highest frequency (count) of deletion segments.

Hint: Subset the `segs.gr` to only rows with `Segment_Mean < -0.3`. 

In [65]:
overlaps <- find_overlaps(tiles, segs.gr) %>%
    as_tibble() %>%
    filter(Segment_Mean < -0.3) %>%
    group_by(seqnames, start, end) %>%
    summarize(count = n()) %>%
    arrange(desc(count)) %>%
    head(1)

overlaps

`summarise()` has grouped output by 'seqnames', 'start'. You can override using
the `.groups` argument.


seqnames,start,end,count
<fct>,<int>,<int>,<int>
16,79000001,80000000,620


# Problem 3: Reading and annotating genomic variants (16 points total)

### Preparation

In [81]:
vcfFile <- "GIAB_highconf_v.3.3.2.vcf.gz"

### a. Load variant data from VCF file `GIAB_highconf_v.3.3.2.vcf.gz` for `chr8:128,700,000-129,000,000`. (4 points)
Note: use genome build `hg19`.

In [83]:
myGRange <- GRanges(seqnames = "8", ranges = IRanges(start = 128700000, end = 129000000))
vcf.param <- ScanVcfParam(which = myGRange)

vcf <- readVcf(vcfFile, genome = "hg19", param = vcf.param)

### b. Combine the fields of the VCF genotype information into a table. (4 points)
You may use your choice of data objects (e.g. `data.frame`).

In [98]:
genoData <- bind_cols(lapply(geno(vcf), as.data.frame))
colnames(genoData) <- rownames(geno(header(vcf)))
head(genoData)

New names:
• `HG001` -> `HG001...1`
• `HG001` -> `HG001...2`
• `HG001` -> `HG001...3`
• `HG001` -> `HG001...4`
• `HG001` -> `HG001...5`
• `HG001` -> `HG001...6`
• `HG001` -> `HG001...7`
• `HG001` -> `HG001...8`


,GT,DP,GQ,ADALL,AD,IGT,IPS,PS
,<chr>,<int>,<int>,<named list>,<named list>,<chr>,<chr>,<chr>
rs6984323,1|1,765,583,"1, 332","0, 315",1/1,.,PATMAT
rs4478537,0|1,544,813,"103, 124","135, 172",0/1,.,PATMAT
rs34141920,0|1,523,222,"132, 121","132, 121",0/1,.,PATMAT
rs17772814,1|0,695,1503,"143, 158","196, 199",0/1,.,PATMAT
rs77977256,1|0,642,685,"154, 157","160, 166",0/1,.,PATMAT
8:128715845_AT/A,0|1,368,99,"66, 91","66, 91",0/1,.,PATMAT


### c. Retrieve the following information at chr8:128747953. (8 points)
Print out the SNP ID (i.e. "rs ID"), reference base (`REF`), alterate base (`ALT`), genotype (`GT`), depth (`DP`), allele depth (`ADALL`), phase set (`PS`).

Hints: 

  i. `REF` and `ALT` are in the output of `rowRanges(vcf)`. See Section `3a` in `Lecture16_VariantCalls.ipynb` 
  ii. To get the sequence of `DNAString`, use `as.character(x)`.  
  ii. To get the sequence of `DNAStringSet`, use `as.character(unlist(x))`. 
  iii. To expand a list of information for `geno`, use `unlist(x)`.  

  

In [ ]:
# Method 1 - using the dataframe created in b
vcf_df <- rowRanges(vcf) %>%
    as.data.frame()
vcf_df <- mutate(vcf_df, rs_id = rownames(vcf_df))
genoData <- mutate(genoData, rs_id = rownames(genoData))
vcf_df %>%
    inner_join(genoData, by = "rs_id") %>%
    filter(start == 128747953) %>%
    dplyr::select(rs_id, REF, ALT, GT, DP, ADALL, PS)

rs_id,REF,ALT,GT,DP,ADALL,PS
<chr>,<chr>,<I<list>>,<chr>,<int>,<named list>,<chr>
rs3824120,G,T,0|1,461,"105, 94",PATMAT


In [ ]:
# Method 2 - read variants only at that location
myGRange <- GRanges(seqnames = "8", ranges = IRanges(start = 128747953, end = 128747953))
vcf.param <- ScanVcfParam(which = myGRange)
vcf <- readVcf(vcfFile, genome = "hg19", param = vcf.param)
print(paste("SNP ID:", names(rowRanges(vcf))))
print(paste("REF:", rowRanges(vcf)$REF))
print(paste("ALT:", as.character(unlist(rowRanges(vcf)$ALT))))
print(paste("GT:", geno(vcf)$GT))
print(paste("DP:", geno(vcf)$DP))
print("ADALL:")
print(unlist(geno(vcf)$ADALL))
print(paste("PS:", geno(vcf)$PS))

[1] "SNP ID: rs3824120"
[1] "REF: G"
[1] "ALT: T"
[1] "GT: 0|1"
[1] "DP: 461"
[1] "ADALL:"
[1] 105  94
[1] "PS: PATMAT"
